<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<h1>ATM 303 LabJack sampling and plotting lesson</h1>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="http://matplotlib.org/_images/date_demo.png" alt="METAR" style="height: 300px;"></div>

### Steps
1. <a href="#sampling">Sample from the LabJack device</a>
1. <a href="#plotting">Plot data</a>

## Sample LabJack code here was adapted to make this notebook
http://www.physics.hmc.edu/courses/p057/pmwiki/pmwiki.php/Software/SampleLabJackCode

<a name="sampling"></a>
### Read a single value

In [1]:
import u3            # load the u3 module
d = u3.U3()          # create a u3 device controller
d.configU3()         # configure it
print d.getAIN( 0 )  # read analog input channel 0 and print the result

SyntaxError: invalid syntax (<ipython-input-1-bfc98b94259f>, line 4)

### Streaming data for an indefinite time

The following Python program will sample one or more analog input channels of the LabJack at a specified sampling frequency for an indefinite time. By editing the first few variable declarations at the top of the file, you may set

- the number of channels
- the sampling frequency
- the filename to which to write the data (in text form)
- the column names for the data (if you wish; defaults will be provided if you don't)

Once started, the program will run indefinitely. Press Control-C to stop it.

Caution: if the program reports an error during the run, you should not trust the data. This tends to happen when you ask the LabJack to record samples near its rated capacity. See the LabJack site for details.

In [ ]:
# Peter N. Saeta, 2013 November 11

# This code uses the LabJack to record 1 or more voltages at a
# regular cadence. It runs until you stop it with Ctrl-C.
# Edit the values below (above the import u3 statement)
# to match your requirements. Note that the Resolution parameter
# sets the accuracy of the data. The smaller the number, the better
# the accuracy, but the slower the sampling rate must be. See
# http://labjack.com/support/u3/users-guide/3.2 for details.

NumChannels = 2
# The number of times each second that each channel will be sampled
SampleFrequency = 10000
Resolution = 2
# Where the data will be written
filename = 'data.txt'
# Put a list of strings with the names you would like associated with
# chan0, chan1, ... Example: wavenames = ['sync', 'V_in', 'V_out']
# If left blank, it will be filled with ['wave0', 'wave1', ...]
wavenames = []

import u3
import time

# Prepare the u3 interface for streaming

d = u3.U3()        # initialize the interface; assumes a single U3 is plugged in to a USB port
d.configU3()    # set default configuration
d.configIO( FIOAnalog = 1 )        # ask for analog inputs

# In case the stream was left running from a previous execution
try: d.streamStop()
except: pass


d.streamConfig( NumChannels = NumChannels,
    PChannels = range(NumChannels),
    NChannels = [ 31 for x in range(NumChannels) ],
    Resolution = Resolution,
    SampleFrequency = SampleFrequency )

#d.packetsPerRequest = 1000

# Try to measure a data set.
def measure():
    try:
        for r in d.streamData():
            if r is not None:
                if r['errors'] or r['numPackets'] != d.packetsPerRequest or r['missed']:
                    print "error: errors = '%s', numpackets = %d, missed = '%s'" % (r['errors'], r['numPackets'], r['missed'])
                break
    finally:
        pass
    return r

# Write a set of data to the file
def writeData( r ):
    chans = [ r['AIN%d' % (n)] for n in range(NumChannels) ]
    for i in range(len(chans[0])):
        f.write( "\t".join( ['%.6f' % c[i] for c in chans] ) + '\n' )

with open(filename, 'w') as f:
    f.write( "frequency=%d\n" % SampleFrequency)
    if wavenames == []:
        wavenames = ['wave%d' % n for n in range(NumChannels)]
    f.write( '\t'.join(wavenames) + '\n')

# start the stream
d.streamStart()
loop = 0

try:
    start = time.time()
    while True:
        with open(filename, 'a') as f:
            writeData( measure() )
        loop += 1
        print( "[%.4d %.2f s]" % (loop, time.time() - start))
finally:
    d.streamStop()

<a name="plotting"></a>
# Plotting code 

<a href="#top">Top</a>
<hr style="height:2px;">